In [2]:
#@title setup

try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

# Install if in Colab
if IN_COLAB:
    %pip install transformer_lens
    %pip install circuitsvis
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs  # noqa


from circuitsvis.attention import attention_heads
from fancy_einsum import einsum
from functools import partial
from IPython.display import HTML, IFrame
from jaxtyping import Float
from os import environ
from transformer_lens import ActivationCache, HookedTransformer
from typing import List, Optional, Union
import circuitsvis as cv
import einops
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objs as go
import plotly
import torch
import tqdm.auto as tqdm
import transformer_lens
import transformer_lens.utils as utils
import itertools


Running as a Colab notebook
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.9/176.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.1
    Uninstalling typeguard-4.4.1:
      Successfully uninstalled typeguard-4.4.1
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERRO

In [3]:
#@title Load Model

# plotly.offline.init_notebook_mode()

torch.set_grad_enabled(False)

DEVICE = utils.get_device()
print(f'{DEVICE=}')

PRETRAINED_MODEL = 'gpt2-small'  # gpt2-small or gpt2-medium
print(f'{PRETRAINED_MODEL=}')

model = transformer_lens.HookedTransformer.from_pretrained(
    PRETRAINED_MODEL, center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True, device=DEVICE)

print("print model structure", model)

DEVICE=device(type='cpu')
PRETRAINED_MODEL='gpt2-small'


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer
print model structure HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint

In [3]:
#@title Experiment on tokenization

prompt = "123+456="

# for i in range(len(prompt)):
# add space in various places in the prompt
for add_space_indices in itertools.product([True, False], repeat=len(prompt)):
  prompt_with_space = prompt
  for i, bool_add_space in zip(reversed(range(len(prompt))), reversed(add_space_indices)):
    if bool_add_space:
      prompt_with_space = prompt_with_space[:i] + " " + prompt_with_space[i:]
  print(f'\n{prompt_with_space=}')
  # utils.test_prompt(prompt=prompt_with_space, answer=' 579', model=model, top_k=1)
  utils.test_prompt(prompt=prompt_with_space, answer='579', model=model, top_k=1, prepend_space_to_answer=False)
  print("\n")
  utils.test_prompt(prompt=prompt_with_space, answer=' 579', model=model, top_k=1, prepend_space_to_answer=False)



# for p in prompts:
#   print(f"\nprompt=|{p}|, answer=| 579|")
#



prompt_with_space=' 1 2 3 + 4 5 6 ='
Tokenized prompt: ['<|endoftext|>', ' 1', ' 2', ' 3', ' +', ' 4', ' 5', ' 6', ' =']
Tokenized answer: ['579']


Performance on answer token:
Rank: 28357    Logit: -0.51 Prob:  0.00% Token: |579|

Top 0th token. Logit: 12.98 Prob:  9.37% Token: | 7|


Ranks of the answer tokens: [('579', 28357)]



Tokenized prompt: ['<|endoftext|>', ' 1', ' 2', ' 3', ' +', ' 4', ' 5', ' 6', ' =']
Tokenized answer: [' 5', '79']


...

Performance on answer token:
Rank: 1025     Logit:  5.06 Prob:  0.01% Token: | 5|

Top 0th token. Logit: 10.69 Prob:  3.82% Token: |123|


Performance on answer token:
Rank: 60       Logit:  9.65 Prob:  0.37% Token: |79|

Top 0th token. Logit: 11.48 Prob:  2.30% Token: |
|


Ranks of the answer tokens: [(' 5', 1025), ('79', 60)]

top results:

```


Tokenized prompt: ['<|endoftext|>', ' 1', ' 2', ' 3', ' +', ' 45', ' 6', '=']
Tokenized answer: [' 5', '79']
Performance on answer token:
Rank: 2        Logit: 11.92 Prob:  1.28% Token: | 5|
Top 0th token. Logit: 12.34 Prob:  1.94% Token: | 1|
Performance on answer token:
Rank: 309      Logit:  6.43 Prob:  0.03% Token: |79|
Top 0th token. Logit: 12.09 Prob:  9.01% Token: |
|
Ranks of the answer tokens: [(' 5', 2), ('79', 309)]

Tokenized prompt: ['<|endoftext|>', ' 12', ' 3', ' +', ' 4', ' 5', ' 6', '=']
Tokenized answer: [' 5', '79']
Performance on answer token:
Rank: 3        Logit: 11.82 Prob:  2.44% Token: | 5|
Top 0th token. Logit: 12.56 Prob:  5.13% Token: |
|
Performance on answer token:
Rank: 1747     Logit:  4.54 Prob:  0.00% Token: |79|
Top 0th token. Logit: 12.84 Prob:  9.16% Token: |
|
Ranks of the answer tokens: [(' 5', 3), ('79', 1747)]

Tokenized prompt: ['<|endoftext|>', '1', ' 2', ' 3', ' +', ' 45', ' 6', '=']
Tokenized answer: [' 5', '79']
Performance on answer token:
Rank: 3        Logit: 11.91 Prob:  1.23% Token: | 5|
Top 0th token. Logit: 12.24 Prob:  1.71% Token: | 1|
Performance on answer token:
Rank: 283      Logit:  6.63 Prob:  0.04% Token: |79|
Top 0th token. Logit: 12.05 Prob:  8.38% Token: |
|
Ranks of the answer tokens: [(' 5', 3), ('79', 283)]

```



In [5]:
#@title acceptable token on the answers "579" and some wrong answer tokens

# ['<|endoftext|>', ' 1', ' 2', ' 3', ' +', ' 45', ' 6', '=']
input_token = " 1 2 3 + 45 6="
answer_token = " 579"

output_logit, cache = model.run_with_cache(input_token)
print(f'{output_logit.shape=}')

print('acceptable tokens')
print(f'|5|   token: {model.to_single_token("5")} \t logit: {output_logit[0, -1, model.to_single_token("5")]}')
print(f'|57|  token: {model.to_single_token("57")} \t logit: {output_logit[0, -1, model.to_single_token("57")]}')
print(f'|579| token: {model.to_single_token("579")} \t logit: {output_logit[0, -1, model.to_single_token("579")]}')
print(f'| 5|  token: {model.to_single_token(" 5")} \t logit: {output_logit[0, -1, model.to_single_token(" 5")]}')
print(f'| 57| token: {model.to_single_token(" 57")} \t logit: {output_logit[0, -1, model.to_single_token(" 57")]}')

print('top wrong tokens')
print(f'|1|   token: {model.to_single_token("1")} \t logit: {output_logit[0, -1, model.to_single_token("1")]}')
print(f'| 1|  token: {model.to_single_token(" 1")} \t logit: {output_logit[0, -1, model.to_single_token(" 1")]}')

print(f'|12|  token: {model.to_single_token("12")} \t logit: {output_logit[0, -1, model.to_single_token("12")]}')
print(f'| 7|  token: {model.to_single_token(" 7")} \t logit: {output_logit[0, -1, model.to_single_token(" 7")]}')

right_token_str = ['|5|', '|57|', '|579|', '| 5|', '| 57|']
right_tokens=[20, 3553, 41734, 642, 7632]
wrong_token_str = ['|1|', '| 1|', '|12|', '| 7|']
wrong_tokens=[16, 352, 1065, 767]


accumulated_residual, labels = cache.accumulated_resid(
    layer=-1, incl_mid=True, pos_slice=-1, return_labels=True, apply_ln=True
)
accumulated_residual_logits = accumulated_residual @ model.unembed.W_U + model.unembed.b_U
fig = go.Figure()
for token_str, t in zip(right_token_str, right_tokens):
  fig.add_trace(go.Scatter(
    x=np.arange(model.cfg.n_layers * 2 + 1) / 2,
    y=utils.to_numpy(accumulated_residual_logits[:, 0, t]),
    text=labels,
    name=token_str,
        # line=dict(
        #     color='blue',
        #     # dash ='dot',
        #     # width=3.5,
        # )
  ))
for token_str, t in zip(wrong_token_str, wrong_tokens):
  fig.add_trace(go.Scatter(
    x=np.arange(model.cfg.n_layers * 2 + 1) / 2,
    y=utils.to_numpy(accumulated_residual_logits[:, 0, t]),
    text=labels,
    name=token_str,
    # color='royalblue',
    line=dict(
            # color='red',
            dash ='dot',
            # width=3.5,
        )
  ))
fig.show()


output_logit.shape=torch.Size([1, 8, 50257])
acceptable tokens
|5|   token: 20 	 logit: 11.259031295776367
|57|  token: 3553 	 logit: 9.810115814208984
|579| token: 41734 	 logit: 4.527767658233643
| 5|  token: 642 	 logit: 11.922718048095703
| 57| token: 7632 	 logit: 10.250670433044434
top wrong tokens
|1|   token: 16 	 logit: 11.585814476013184
| 1|  token: 352 	 logit: 12.338008880615234
|12|  token: 1065 	 logit: 11.043591499328613
| 7|  token: 767 	 logit: 11.680510520935059


In [39]:
stack_head_result, labels = cache.stack_head_results(
    layer=-1, pos_slice=-1, return_labels=True, apply_ln=True
)
stack_head_result = stack_head_result @ model.unembed.W_U + model.unembed.b_U
stack_head_result = stack_head_result.view(12, 12, -1)

figs = make_subplots(rows=2, cols=5, subplot_titles=right_token_str+wrong_token_str)
i = 1
for token_str, t in zip(right_token_str, right_tokens):
  subfig = px.imshow(
      utils.to_numpy(stack_head_result[:,:, t]),
      labels={"x": "Head", "y": "Layer"},
  )
  for trace in subfig.data:
    figs.add_trace(trace, row=1, col=i)
  i = i + 1

# wrong_token_fig = make_subplots(rows=1, cols=4, subplot_titles=wrong_token_str)
i = 1
for token_str, t in zip(wrong_token_str, wrong_tokens):
  subfig = px.imshow(
      utils.to_numpy(stack_head_result[:,:, t]),
      labels={"x": "Head", "y": "Layer"},
  )
  for trace in subfig.data:
    figs.add_trace(trace, row=2, col=i)
  i = i + 1
figs.show()


